Environment && Library

In [85]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"
import tensorflow as tf
import numpy as np

Configuration

In [86]:
augmentation = False
quant = True
exit = 4
if(int(exit) > 4 or int(exit) < 1):
    exit = 0
if(augmentation == True):
    model = tf.keras.models.load_model("/home/x/Desktop/augmentation_functionAPI/")
    tittle = "aug_" + str(exit)
elif(augmentation == False):
    model = tf.keras.models.load_model("/home/x/Desktop/Non_augmentation_functionAPI/")
    tittle = "non_" + str(exit)

if(int(exit) < 5 and int(exit) > 0):
    model = tf.keras.Model(inputs = model.layers[1].input, outputs = model.layers[7 + int(exit)].output)
elif(int(exit) > 4 or int(exit) < 1):
    model = tf.keras.Model(inputs = model.layers[1].input, outputs = [model.layers[7 + 1].output, model.layers[7 + 2].output, model.layers[7 + 3].output, model.layers[7 + 4].output])
    
model.summary()
print(model.dtype)

Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 28, 28, 1)]       0         
                                                                 
 C1 (Conv2D)                 (None, 24, 24, 6)         156       
                                                                 
 S2 (AveragePooling2D)       (None, 12, 12, 6)         0         
                                                                 
 C3 (Conv2D)                 (None, 8, 8, 16)          2416      
                                                                 
 S4 (AveragePooling2D)       (None, 4, 4, 16)          0         
                                                                 
 Flatten (Flatten)           multiple                  0         
                                                                 
 F5 (Dense)                  (None, 120)               308

Dataset

In [87]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train.reshape(-1, 28 ,28, 1).astype(np.float32) / 255.0, x_test.reshape(-1, 28 ,28, 1).astype(np.float32) / 255.0
if(augmentation == True):
    train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range= 30, width_shift_range= 0.2, height_shift_range= 0.2, zoom_range= 0.2, data_format= 'channels_last', dtype= np.float32)
elif(augmentation == False):
    train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range= 0, width_shift_range= 0, height_shift_range= 0, zoom_range= 0, data_format= 'channels_last', dtype= np.float32)
train_gen.fit(x_train)
train_generator = train_gen.flow(x_train, y_train, batch_size= 10000, shuffle=True, seed=2, save_to_dir= None)

float32

In [88]:
if(quant == False):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()

    # Save the unquantized/float model:
    tflite_model_path = "/home/x/Desktop/" + tittle + "_float32.tflite"
    with open(tflite_model_path, "wb") as f:
        f.write(tflite_model)

    interpreter = tf.lite.Interpreter(model_content=tflite_model)
    print('input: ', interpreter.get_input_details()[0]['dtype'])
    print('output: ', interpreter.get_output_details()[0]['dtype'])

uint8

In [89]:
if(quant == True):
  def representative_data_gen():
    x,y = train_generator.next()
    for input_value in tf.data.Dataset.from_tensor_slices(x).batch(1).take(100):
      yield [input_value]

  converter = tf.lite.TFLiteConverter.from_keras_model(model)
  converter.optimizations = [tf.lite.Optimize.DEFAULT]
  converter.representative_dataset = representative_data_gen
  # Ensure that if any ops can't be quantized, the converter throws an error
  converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
  # Set the input and output tensors to uint8 (APIs added in r2.3)
  converter.inference_input_type = tf.uint8
  converter.inference_output_type = tf.uint8
  tflite_model_quant = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpgddc1ony/assets


INFO:tensorflow:Assets written to: /tmp/tmpgddc1ony/assets
/home/x/.local/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-04-15 01:49:30.864570: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-04-15 01:49:30.864592: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-04-15 01:49:30.864697: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpgddc1ony
2023-04-15 01:49:30.865530: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2023-04-15 01:49:30.865546: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: /tmp/tmpgddc1ony
2023-04-15 01:49:30.868106: I tensorflow/cc/saved_model/loader.cc:228] 

In [90]:
if(quant == True):
  # Save the quantized model:
  tflite_model_quant_path = "/home/x/Desktop/" + tittle + "_uint8.tflite"
  with open(tflite_model_quant_path, "wb") as f:
      f.write(tflite_model_quant)
  interpreter = tf.lite.Interpreter(model_content = tflite_model_quant)
  print('input: ', interpreter.get_input_details()[0]['dtype'])
  print('output: ', interpreter.get_output_details()[0]['dtype'])

input:  <class 'numpy.uint8'>
output:  <class 'numpy.uint8'>
